<a href="https://colab.research.google.com/github/huntersarnelli/x-ray_images/blob/main/V2_of_Xray_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip -d /content/chest_xray


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
... resuming from 92274688 bytes (2371090747 bytes left) ...
100% 2.29G/2.29G [01:41<00:00, 24.4MB/s]
100% 2.29G/2.29G [01:41<00:00, 23.3MB/s]


In [ ]:
!ls /content/chest_xray


chest_xray


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Now we set up the automation based on the path

base_dir = '/content/chest_xray/chest_xray'

train_dir = os.path.join(base_dir,'train')

test_dir = os.path.join(base_dir,'test')

validation_dir = os.path.join(base_dir,'val')




In [ ]:
#Now we build the data generator for the training data

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range =0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

validation_generator = val_test_datagen.flow_from_directory(validation_dir,
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

test_generator = val_test_datagen.flow_from_directory(test_dir,
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')

])

model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Adam optimizer with a learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



history = model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps=50,
                    callbacks=[lr_scheduler],
                    verbose=2)



Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 - 48s - 484ms/step - accuracy: 0.7806 - loss: 0.6049 - val_accuracy: 0.5000 - val_loss: 19.1925 - learning_rate: 0.0010
Epoch 2/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 - 23s - 232ms/step - accuracy: 0.8450 - loss: 0.3672 - val_accuracy: 0.5000 - val_loss: 15.5677 - learning_rate: 0.0010
Epoch 3/15
100/100 - 13s - 131ms/step - accuracy: 0.8828 - loss: 0.2951 - val_accuracy: 0.5000 - val_loss: 5.5700 - learning_rate: 0.0010
Epoch 4/15
100/100 - 26s - 258ms/step - accuracy: 0.8903 - loss: 0.2813 - val_accuracy: 0.5000 - val_loss: 7.6591 - learning_rate: 0.0010
Epoch 5/15
100/100 - 24s - 240ms/step - accuracy: 0.9000 - loss: 0.2564 - val_accuracy: 0.5000 - val_loss: 3.0181 - learning_rate: 0.0010
Epoch 6/15
100/100 - 13s - 131ms/step - accuracy: 0.8803 - loss: 0.2811 - val_accuracy: 0.5000 - val_loss: 6.0674 - learning_rate: 0.0010
Epoch 7/15
100/100 - 26s - 258ms/step - accuracy: 0.8943 - loss: 0.2567 - val_accuracy: 0.5000 - val_loss: 35.8226 - learning_rate: 0.0010
Epoch 8/15
100/100 - 24s - 239ms/step - accuracy: 0.9095 - loss: 0.2462 - val_accuracy: 0.5000 - val_loss: 3.4073 - learning_rate: 1.0000e-04
Epoch 9/15
100/100 - 13s - 134ms/step -

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print(f"Test accuracy: {test_acc*100:.2f}%")


50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 96ms/step - accuracy: 0.9032 - loss: 0.2869 
Test accuracy: 89.26%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save('/content/drive/My Drive/my_model_x-ray.h5')
